In [12]:
pip install geopandas

Note: you may need to restart the kernel to use updated packages.


In [13]:
import pandas as pd
import geopandas as gpd
import numpy as np
from shapely.geometry import MultiPolygon, Point, LineString
import plotly.graph_objects as go
import datetime
from scipy.ndimage import gaussian_filter1d
import requests
import zipfile
import os
import matplotlib.pyplot as plt


def download_ukraine_border():
    url = "https://geodata.ucdavis.edu/gadm/gadm4.1/shp/gadm41_UKR_shp.zip"
    local_zip = "gadm41_UKR_shp.zip"

    if not os.path.exists("gadm_ukraine/gadm41_UKR_0.shp"):
        os.makedirs("gadm_ukraine", exist_ok=True)
        response = requests.get(url)
        with open(local_zip, "wb") as f:
            f.write(response.content)
        with zipfile.ZipFile(local_zip, 'r') as zip_ref:
            zip_ref.extractall("gadm_ukraine")
        os.remove(local_zip)

    return gpd.read_file("gadm_ukraine/gadm41_UKR_0.shp")


In [14]:

def generate_precise_border_nodes(ukraine_border, num_nodes=20000):
    """
    Generate highly precise border nodes using advanced sampling techniques
    """
    border_geom = ukraine_border.geometry.iloc[0]

    if border_geom.geom_type == 'MultiPolygon':
        border_geom = max(border_geom.geoms, key=lambda poly: poly.area)

    if border_geom.geom_type == 'Polygon':
        exterior_coords = list(border_geom.exterior.coords)
    else:
        raise ValueError(f"Unexpected geometry type: {border_geom.geom_type}")

    border_line = LineString(exterior_coords)
    border_length = border_line.length
    border_nodes = [border_line.interpolate(float(i * border_length) / num_nodes) for i in range(num_nodes)]

    frontline_nodes = pd.DataFrame({
        'latitude': [point.y for point in border_nodes],
        'longitude': [point.x for point in border_nodes]
    })

    return frontline_nodes


In [15]:
def load_acled_data():
    """
    Load and preprocess ACLED dataset
    """
    acled = pd.read_csv('ACLED_Ukraine_Reduced.csv')
    acled['event_date'] = pd.to_datetime(acled['event_date'])
    acled = acled[acled['event_date'] >= pd.Timestamp('2022-02-24')]
    acled['week'] = acled['event_date'].dt.to_period('W').apply(lambda r: r.start_time)

    current_date = acled['event_date'].max()
    acled['age_weeks'] = (current_date - acled['event_date']).dt.days / 7
    acled['weight'] = np.exp(-0.2 * acled['age_weeks'])

    return acled


In [16]:

def compute_weekly_frontline(acled, frontline_nodes, ukraine_border,
                             decay_lambda=0.2, search_radius=0.75):
    """
    Compute weekly frontline positions based on ACLED data
    """
    weeks = sorted(acled['week'].unique())
    node_history = []
    current_nodes = frontline_nodes.copy()

    # Convert border to proper geometry for contains check
    border_geom = ukraine_border.geometry.iloc[0]
    if border_geom.geom_type == 'MultiPolygon':
        border_geom = max(border_geom.geoms, key=lambda poly: poly.area)

    for week in weeks:
        print(f"Processing week: {week}")
        weekly_points = acled[acled['week'] <= week].copy()

        new_lat, new_lon = [], []

        for _, node in current_nodes.iterrows():
            distances = np.sqrt((weekly_points['latitude'] - node['latitude']) ** 2 +
                                (weekly_points['longitude'] - node['longitude']) ** 2)
            nearby = weekly_points[distances < search_radius]

            if not nearby.empty:
                offset_lat = ((nearby['latitude'] - node['latitude']) * nearby['weight']).sum() / nearby['weight'].sum()
                offset_lon = ((nearby['longitude'] - node['longitude']) * nearby['weight']).sum() / nearby[
                    'weight'].sum()

                new_point = Point(node['longitude'] + 0.5 * offset_lon, node['latitude'] + 0.5 * offset_lat)

                # Ensure the new point is within Ukraine's border
                if border_geom.contains(new_point):
                    new_lat.append(new_point.y)
                    new_lon.append(new_point.x)
                else:
                    new_lat.append(node['latitude'])
                    new_lon.append(node['longitude'])
            else:
                new_lat.append(node['latitude'])
                new_lon.append(node['longitude'])

        # Apply smoothing to the new positions
        new_lat = gaussian_filter1d(new_lat, sigma=3)
        new_lon = gaussian_filter1d(new_lon, sigma=3)

        current_nodes['latitude'], current_nodes['longitude'] = new_lat, new_lon
        current_nodes['week'] = week
        node_history.append(current_nodes.copy())

    return node_history



In [17]:

def plot_frontline_with_slider(node_history, acled):
    """
    Create an interactive Plotly map with slider to show frontline progression
    """
    fig = go.Figure()

    for i, week_nodes in enumerate(node_history):
        week_nodes = week_nodes.copy()
        fig.add_trace(go.Scattermapbox(
            lat=week_nodes['latitude'],
            lon=week_nodes['longitude'],
            mode='lines',
            line=dict(width=2, color='red'),
            name=str(week_nodes['week'].iloc[0]),
            visible=(i == 0)
        ))

        week_points = acled[acled['week'] == week_nodes['week'].iloc[0]]
        if not week_points.empty:
            fig.add_trace(go.Scattermapbox(
                lat=week_points['latitude'],
                lon=week_points['longitude'],
                mode='markers',
                marker=dict(size=week_points['weight'] * 25, color='orange', opacity=0.4),
                showlegend=False,
                visible=(i == 0)
            ))

    steps = []
    for i in range(len(node_history)):
        step = dict(method="update", args=[{"visible": [False] * len(fig.data)}])
        line_idx = i * 2
        step["args"][0]["visible"][line_idx] = True
        if line_idx + 1 < len(fig.data):
            step["args"][0]["visible"][line_idx + 1] = True
        step["label"] = str(node_history[i]['week'].iloc[0].date())
        steps.append(step)

    sliders = [dict(active=0, currentvalue={"prefix": "Week: "}, steps=steps)]

    fig.update_layout(
        sliders=sliders,
        mapbox_style="carto-positron",
        mapbox_zoom=5,
        mapbox_center={"lat": 49, "lon": 32},
        height=800
    )

    fig.show()



In [18]:
# 1. Download and load Ukraine border
ukraine = download_ukraine_border()


In [19]:
# 2. Generate precise border nodes
frontline_nodes = generate_precise_border_nodes(ukraine, num_nodes=20000)


In [20]:
# 3. Load ACLED data
acled = load_acled_data()


In [ ]:
# 4. Compute weekly frontline
node_history = compute_weekly_frontline(acled, frontline_nodes, ukraine)


Processing week: 2022-02-21 00:00:00
Processing week: 2022-02-28 00:00:00
Processing week: 2022-03-07 00:00:00
Processing week: 2022-03-14 00:00:00
Processing week: 2022-03-21 00:00:00
Processing week: 2022-03-28 00:00:00
Processing week: 2022-04-04 00:00:00
Processing week: 2022-04-11 00:00:00
Processing week: 2022-04-18 00:00:00
Processing week: 2022-04-25 00:00:00
Processing week: 2022-05-02 00:00:00
Processing week: 2022-05-09 00:00:00
Processing week: 2022-05-16 00:00:00
Processing week: 2022-05-23 00:00:00
Processing week: 2022-05-30 00:00:00
Processing week: 2022-06-06 00:00:00
Processing week: 2022-06-13 00:00:00
Processing week: 2022-06-20 00:00:00
Processing week: 2022-06-27 00:00:00
Processing week: 2022-07-04 00:00:00
Processing week: 2022-07-11 00:00:00
Processing week: 2022-07-18 00:00:00
Processing week: 2022-07-25 00:00:00
Processing week: 2022-08-01 00:00:00
Processing week: 2022-08-08 00:00:00
Processing week: 2022-08-15 00:00:00
Processing week: 2022-08-22 00:00:00
P

In [ ]:
# 5. Plot interactive frontline map
plot_frontline_with_slider(node_history, acled)

In [ ]:
def plot_frontline_with_slider_auto(node_history, acled):
    fig = go.Figure()


    frames = []
    for i, week_nodes in enumerate(node_history):
        week_nodes = week_nodes.copy()

        # Add line for this frame
        line_trace = go.Scattermapbox(
            lat=week_nodes['latitude'],
            lon=week_nodes['longitude'],
            mode='lines',
            line=dict(width=2, color='red'),
            name=f"Frontline - {week_nodes['week'].iloc[0].date()}"
        )

        week_points = acled[acled['week'] == week_nodes['week'].iloc[0]]
        marker_trace = go.Scattermapbox(
            lat=week_points['latitude'],
            lon=week_points['longitude'],
            mode='markers',
            marker=dict(size=week_points['weight'] * 25, color='orange', opacity=0.4),
            showlegend=False
        ) if not week_points.empty else None

        frame_data = [line_trace]
        if marker_trace:
            frame_data.append(marker_trace)

        frames.append(go.Frame(data=frame_data, name=str(week_nodes['week'].iloc[0].date())))

    # Initial data
    fig.add_traces(frames[0].data)

    fig.update(frames=frames)

    fig.update_layout(
        updatemenus=[dict(
            type="buttons",
            showactive=False,
            buttons=[
                dict(label="Play",
                     method="animate",
                     args=[None, {"frame": {"duration": 500, "redraw": True}, "fromcurrent": True}]),
                dict(label="Pause",
                     method="animate",
                     args=[[None], {"frame": {"duration": 0, "redraw": False}, "mode": "immediate"}])
            ],
            x=0.05, y=0, xanchor="left", yanchor="bottom"
        )],
        sliders=[dict(
            steps=[dict(method='animate',
                        args=[[frame.name], {"frame": {"duration": 500, "redraw": True}, "mode": "immediate"}],
                        label=frame.name) for frame in frames],
            currentvalue=dict(prefix="Week: ")
        )],
        mapbox_style="carto-positron",
        mapbox_zoom=5,
        mapbox_center={"lat": 49, "lon": 32},
        height=800
    )

    fig.write_html("frontline_map_FINAL.html")
    webbrowser.open("frontline_map_FINAL.html")

In [ ]:
import webbrowser

In [ ]:
plot_frontline_with_slider_auto(node_history, acled)